In [ ]:
# IMPORT ALL YOUR LIBRARIES
!pip install transformers
!pip install datasets
!pip install accelerate >=0.21.0
!pip install transformers[torch]

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
# from datasets import Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/Sem-Eval-Task10-Dataset/refs/heads/main/final_cleaned_train.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/Sem-Eval-Task10-Dataset/refs/heads/main/final_cleaned_test.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/Sem-Eval-Task10-Dataset/refs/heads/main/final_cleaned_validation.csv"

--2024-10-01 06:11:57--  https://raw.githubusercontent.com/HammadxSaj/Sem-Eval-Task10-Dataset/refs/heads/main/final_cleaned_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12659880 (12M) [text/plain]
Saving to: ‘/kaggle/working/final_cleaned_train.csv’

final_cleaned_train 100%[===================>]  12.07M  --.-KB/s    in 0.1s    

2024-10-01 06:11:57 (112 MB/s) - ‘/kaggle/working/final_cleaned_train.csv’ saved [12659880/12659880]

--2024-10-01 06:11:57--  https://raw.githubusercontent.com/HammadxSaj/Sem-Eval-Task10-Dataset/refs/heads/main/final_cleaned_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com

In [ ]:
df = pd.read_csv('/kaggle/working/final_cleaned_train.csv')
df

,year,month,day,country,title,text,hazard-category,product-category,hazard,product
0,1994,1,7,us,Recall Notification: FSIS-024-94,Date Opened: Date Closed: Name: GERHARD'S NAPA...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage
1,1994,3,10,us,Recall Notification: FSIS-033-94,Date Opened: Date Closed: Name: WIMMER'S MEAT ...,biological,"meat, egg and dairy products",listeria spp,sausage
2,1994,3,28,us,Recall Notification: FSIS-014-94,Date Opened: Date Closed: Name: WILLOW FOODS I...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices
3,1994,4,3,us,Recall Notification: FSIS-009-94,Date Opened: Date Closed: M Name: OSCAR MAYER ...,foreign bodies,"meat, egg and dairy products",plastic fragment,thermal processed pork meat
4,1994,7,1,us,Recall Notification: FSIS-001-94,Date Opened: Date Closed: Name: TYSON FOODS Im...,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast
...,...,...,...,...,...,...,...,...,...,...
5961,2022,7,28,hk,Imported biscuit may contain allergen (peanuts),Imported biscuit may contain allergen (peanuts...,allergens,cereals and bakery products,peanuts and products thereof,biscuits
5962,2022,7,28,us,Danny’s Sub and Pizza Recalls Meat Pizza Produ...,023-2022 High - Class I Produced Without Benef...,fraud,prepared dishes and snacks,inspection issues,pizza
5963,2022,7,29,us,Lyons Magnus Voluntarily Recalls 53 Nutritiona...,"FRESNO, Calif. – – Lyons Magnus LLC (“Lyons Ma...",biological,non-alcoholic beverages,cronobacter spp,non-alcoholic beverages
5964,2022,7,30,us,"Conagra Brands, Inc., Recalls Frozen Beef Prod...",025-2022 High - Class I Misbranding Mislabelin...,allergens,"meat, egg and dairy products",eggs and products thereof,frozen beef products


In [ ]:
df['text'] = df['title'] + " " + df['text']

# Drop unnecessary columns for training
df = df[['text', 'hazard-category', 'product-category', 'hazard', 'product']]

# Drop rows with missing values
df.dropna(inplace=True)

# Split dataset into 5 chunks with ~1000 rows each
chunk_size = 2000
chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

# Save each chunk into a separate CSV file (for ease of handling)
for i, chunk in enumerate(chunks):
    chunk.to_csv(f'/content/dataset_chunk_{i}.csv', index=False)

<ipython-input-4-319fb8271b20>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Label encoders (common across all chunks)
hazard_category_encoder = LabelEncoder()
product_category_encoder = LabelEncoder()
hazard_encoder = LabelEncoder()
product_encoder = LabelEncoder()

# Fit the encoders on the entire dataset (not just the chunks)
df_full = pd.concat(chunks)
hazard_category_encoder.fit(df_full['hazard-category'])
product_category_encoder.fit(df_full['product-category'])
hazard_encoder.fit(df_full['hazard'])
product_encoder.fit(df_full['product'])

# Function to preprocess each chunk
def preprocess_chunk(chunk):
    chunk['hazard-category'] = hazard_category_encoder.transform(chunk['hazard-category'])
    chunk['product-category'] = product_category_encoder.transform(chunk['product-category'])
    chunk['hazard'] = hazard_encoder.transform(chunk['hazard'])
    chunk['product'] = product_encoder.transform(chunk['product'])

    train_texts, test_texts, train_hazard_categories, test_hazard_categories, \
    train_product_categories, test_product_categories, train_hazards, test_hazards, \
    train_products, test_products = train_test_split(
        chunk['text'].tolist(),
        chunk['hazard-category'].tolist(),
        chunk['product-category'].tolist(),
        chunk['hazard'].tolist(),
        chunk['product'].tolist(),
        test_size=0.2, random_state=42
    )

    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

    return train_encodings, test_encodings, train_hazard_categories, test_hazard_categories, \
           train_product_categories, test_product_categories, train_hazards, test_hazards, \
           train_products, test_products


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import torch

class FoodHazardDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, hazard_categories, product_categories, hazards, products):
        self.encodings = encodings
        self.hazard_categories = hazard_categories
        self.product_categories = product_categories
        self.hazards = hazards
        self.products = products

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['hazard_category_labels'] = torch.tensor(self.hazard_categories[idx])
        item['product_category_labels'] = torch.tensor(self.product_categories[idx])
        item['hazard_labels'] = torch.tensor(self.hazards[idx])
        item['product_labels'] = torch.tensor(self.products[idx])
        return item

    def __len__(self):
        return len(self.hazard_categories)


In [ ]:
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn

# Define the number of unique labels for each category
num_hazard_category_labels = len(hazard_category_encoder.classes_)
num_product_category_labels = len(product_category_encoder.classes_)
num_hazard_labels = len(hazard_encoder.classes_)
num_product_labels = len(product_encoder.classes_)

class BertForFoodHazardClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # Classifiers for the four labels
        self.hazard_category_classifier = nn.Linear(config.hidden_size, num_hazard_category_labels)
        self.product_category_classifier = nn.Linear(config.hidden_size, num_product_category_labels)
        self.hazard_classifier = nn.Linear(config.hidden_size, num_hazard_labels)
        self.product_classifier = nn.Linear(config.hidden_size, num_product_labels)

        # Loss function
        self.loss_fct = nn.CrossEntropyLoss()

        # Initialize the weights of the model
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                hazard_category_labels=None, product_category_labels=None,
                hazard_labels=None, product_labels=None):

        # BERT forward pass
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]  # Get the pooled output (CLS token)
        pooled_output = self.dropout(pooled_output)

        # Predict the four labels
        hazard_category_logits = self.hazard_category_classifier(pooled_output)
        product_category_logits = self.product_category_classifier(pooled_output)
        hazard_logits = self.hazard_classifier(pooled_output)
        product_logits = self.product_classifier(pooled_output)

        loss = None
        if hazard_category_labels is not None and product_category_labels is not None \
           and hazard_labels is not None and product_labels is not None:
            # Compute loss for each task
            hazard_category_loss = self.loss_fct(hazard_category_logits, hazard_category_labels)
            product_category_loss = self.loss_fct(product_category_logits, product_category_labels)
            hazard_loss = self.loss_fct(hazard_logits, hazard_labels)
            product_loss = self.loss_fct(product_logits, product_labels)

            # Aggregate the losses
            loss = hazard_category_loss + product_category_loss + hazard_loss + product_loss

        # Return the loss if training, otherwise return logits for inference
        return (loss, hazard_category_logits, product_category_logits, hazard_logits, product_logits) if loss is not None else (hazard_category_logits, product_category_logits, hazard_logits, product_logits)


In [ ]:
from transformers import Trainer, TrainingArguments, AdamW
from sklearn.metrics import accuracy_score, f1_score
import torch

# Define the compute_metrics function to calculate both accuracy and average F1 score across all labels
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions

    # Unpack labels and predictions for each task
    hazard_category_labels = labels[0]
    product_category_labels = labels[1]
    hazard_labels = labels[2]
    product_labels = labels[3]

    hazard_category_preds = preds[0].argmax(-1)
    product_category_preds = preds[1].argmax(-1)
    hazard_preds = preds[2].argmax(-1)
    product_preds = preds[3].argmax(-1)

    # Compute accuracy for each task (can be used separately if needed)
    hazard_category_acc = accuracy_score(hazard_category_labels, hazard_category_preds)
    product_category_acc = accuracy_score(product_category_labels, product_category_preds)
    hazard_acc = accuracy_score(hazard_labels, hazard_preds)
    product_acc = accuracy_score(product_labels, product_preds)

    # Compute F1 score for each task
    hazard_category_f1 = f1_score(hazard_category_labels, hazard_category_preds, average='weighted')
    product_category_f1 = f1_score(product_category_labels, product_category_preds, average='weighted')
    hazard_f1 = f1_score(hazard_labels, hazard_preds, average='weighted')
    product_f1 = f1_score(product_labels, product_preds, average='weighted')

    # Compute average F1 score across all tasks
    avg_f1 = (hazard_category_f1 + product_category_f1 + hazard_f1 + product_f1) / 4

    # Optionally, you can also compute average accuracy across tasks if needed
    avg_acc = (hazard_category_acc + product_category_acc + hazard_acc + product_acc) / 4

    # Return a dictionary with both accuracy and average F1 score
    return {
        'hazard_category_acc': hazard_category_acc,
        'product_category_acc': product_category_acc,
        'hazard_acc': hazard_acc,
        'product_acc': product_acc,
        'avg_accuracy': avg_acc,
        'avg_f1': avg_f1
    }

# Custom DataCollator to handle multiple labels
def data_collator(batch):
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'hazard_category_labels': torch.tensor([x['hazard_category_labels'] for x in batch]),
        'product_category_labels': torch.tensor([x['product_category_labels'] for x in batch]),
        'hazard_labels': torch.tensor([x['hazard_labels'] for x in batch]),
        'product_labels': torch.tensor([x['product_labels'] for x in batch]),
    }

# Training arguments (adjust batch size if necessary)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    warmup_steps=500,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the model
model = BertForFoodHazardClassification.from_pretrained('bert-base-uncased')

for i, chunk in enumerate(chunks):
    print(f"Training on chunk {i + 1} / {len(chunks)}")

    # Load chunk data
    chunk = pd.read_csv(f'/content/dataset_chunk_{i}.csv')

    # Preprocess chunk
    train_encodings, test_encodings, train_hazard_categories, test_hazard_categories, \
    train_product_categories, test_product_categories, train_hazards, test_hazards, \
    train_products, test_products = preprocess_chunk(chunk)

    # Prepare datasets
    train_dataset = FoodHazardDataset(train_encodings, train_hazard_categories, train_product_categories, train_hazards, train_products)
    test_dataset = FoodHazardDataset(test_encodings, test_hazard_categories, test_product_categories, test_hazards, test_products)

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        optimizers=(AdamW(model.parameters(), lr=5e-5), None),
        data_collator=data_collator
    )

    # Train on the chunk
    trainer.train()

    # Save the model after training on each chunk
    model.save_pretrained(f'/content/saved_model_chunk_{i}')

    # Load the trained model weights before continuing to the next chunk
    model = BertForFoodHazardClassification.from_pretrained(f'/content/saved_model_chunk_{i}')


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForFoodHazardClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['hazard_category_classifier.bias', 'hazard_category_classifier.weight', 'hazard_classifier.bias', 'hazard_classifier.weight', 'product_category_classifier.bias', 'product_category_classifier.weight', 'product_classifier.bias', 'product_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training on chunk 1 / 3


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Hazard Category Acc,Product Category Acc,Hazard Acc,Product Acc,Avg Accuracy,Avg F1
1,14.622200,14.187063,0.672500,0.370000,0.170000,0.072500,0.321250,0.213960
2,10.957100,11.679523,0.832500,0.410000,0.400000,0.080000,0.430625,0.337185
3,9.456600,10.041700,0.880000,0.552500,0.572500,0.105000,0.527500,0.457460


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Hazard Category Acc,Product Category Acc,Hazard Acc,Product Acc,Avg Accuracy,Avg F1
1,11.570200,10.669070,0.880000,0.452500,0.590000,0.072500,0.498750,0.435557
2,9.636800,9.939833,0.902500,0.487500,0.637500,0.115000,0.535625,0.472460
3,9.272000,9.345922,0.897500,0.552500,0.687500,0.120000,0.564375,0.509445


Training on chunk 3 / 3


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Hazard Category Acc,Product Category Acc,Hazard Acc,Product Acc,Avg Accuracy,Avg F1
1,11.071900,10.744911,0.870558,0.467005,0.598985,0.060914,0.499365,0.451175
2,9.912600,10.182871,0.883249,0.505076,0.639594,0.076142,0.526015,0.480968
3,8.927700,9.614098,0.890863,0.538071,0.654822,0.101523,0.546320,0.495754


In [ ]:
# Evaluate the model on the last chunk
results = trainer.evaluate()
print(results)


{'eval_loss': 9.614097595214844, 'eval_hazard_category_acc': 0.8908629441624365, 'eval_product_category_acc': 0.5380710659898477, 'eval_hazard_acc': 0.6548223350253807, 'eval_product_acc': 0.10152284263959391, 'eval_avg_accuracy': 0.5463197969543147, 'eval_avg_f1': 0.4957535725683053, 'eval_runtime': 11.4023, 'eval_samples_per_second': 34.554, 'eval_steps_per_second': 4.385, 'epoch': 3.0}


In [ ]:
import torch
import numpy as np
import pandas as pd

# Load the test CSV file
test_df = pd.read_csv('/kaggle/working/final_cleaned_validation.csv')

# Create a list to store the concatenated title and text for each row
test_texts = (test_df['title'] + ' ' + test_df['text']).tolist()

# Tokenize all the texts in the test file
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Number of chunks used in training
num_chunks = len(chunks)

# Lists to store the logits from all chunks for each task
hazard_category_logits_list = []
product_category_logits_list = []
hazard_logits_list = []
product_logits_list = []

# Iterate over each chunk and load the corresponding model
for i in range(num_chunks):
    print(f"Loading model from chunk {i + 1}")

    # Load the trained model from this chunk
    model = BertForFoodHazardClassification.from_pretrained(f'/content/saved_model_chunk_{i}')
    model.eval()  # Set the model to evaluation mode

    # Get predictions for the entire test set
    with torch.no_grad():
        outputs = model(**test_encodings)

    # Append the logits to the corresponding lists
    hazard_category_logits_list.append(outputs[0].numpy())
    product_category_logits_list.append(outputs[1].numpy())
    hazard_logits_list.append(outputs[2].numpy())
    product_logits_list.append(outputs[3].numpy())

# Convert lists to arrays for easier manipulation
hazard_category_logits_array = np.array(hazard_category_logits_list)
product_category_logits_array = np.array(product_category_logits_list)
hazard_logits_array = np.array(hazard_logits_list)
product_logits_array = np.array(product_logits_list)

# Average the logits across all chunks (ensemble method: averaging)
hazard_category_avg_logits = np.mean(hazard_category_logits_array, axis=0)
product_category_avg_logits = np.mean(product_category_logits_array, axis=0)
hazard_avg_logits = np.mean(hazard_logits_array, axis=0)
product_avg_logits = np.mean(product_logits_array, axis=0)

# Apply argmax to get the final predicted class indices
hazard_category_pred = np.argmax(hazard_category_avg_logits, axis=-1)
product_category_pred = np.argmax(product_category_avg_logits, axis=-1)
hazard_pred = np.argmax(hazard_avg_logits, axis=-1)
product_pred = np.argmax(product_avg_logits, axis=-1)

# Decode the predicted labels using the label encoders
hazard_category_labels = hazard_category_encoder.inverse_transform(hazard_category_pred)
product_category_labels = product_category_encoder.inverse_transform(product_category_pred)
hazard_labels = hazard_encoder.inverse_transform(hazard_pred)
product_labels = product_encoder.inverse_transform(product_pred)

# Create a DataFrame for the predictions
output_df = pd.DataFrame({
    'hazard-category': hazard_category_labels,
    'product-category': product_category_labels,
    'hazard': hazard_labels,
    'product': product_labels
})

# Add a column for serial numbers starting from 0 (without a header)
output_df.index.name = None  # This will ensure the index doesn't have a name in the CSV file

# Save the output DataFrame to a new CSV file with no header for the index column
output_df.to_csv('test_predictions.csv', index=True, index_label=None, header=False)


Loading model from chunk 1
Loading model from chunk 2
Loading model from chunk 3


In [ ]:
import pandas as pd

# Load the dataframe
df = pd.read_csv('Food_Hazard_Detection.csv')

# Assign new headings for the columns
df.columns = ['extra', 'hazard-category', 'product-category', 'hazard', 'product']

# Move the data and the original headers down by 1 row
df_shifted = pd.concat([pd.DataFrame([df.columns], columns=df.columns), df], ignore_index=True)

# Drop the numbers column ('extra'), as the DataFrame's index is already handling row identifiers
df_shifted = df_shifted.drop('extra', axis=1)

# Reset column names for the dataframe
df_shifted.columns = ['hazard-category', 'product-category', 'hazard', 'product']


#assign the 1st row values 'haha', 'haha2', 'haha3', 'haha4'
df_shifted.loc[0] = ['biological', 'meat, egg and dairy products', 'listeria monocytogenes', 'chicken based products']
df_shifted


,hazard-category,product-category,hazard,product
0,biological,"meat, egg and dairy products",listeria monocytogenes,chicken based products
1,biological,"meat, egg and dairy products",escherichia coli,ground beef
2,biological,"meat, egg and dairy products",escherichia coli,cheese
3,allergens,ices and desserts,peanuts and products thereof,ice cream
4,foreign bodies,"meat, egg and dairy products",metal fragment,chicken based products
...,...,...,...,...
560,allergens,fruits and vegetables,peanuts and products thereof,cakes
561,allergens,"cocoa and cocoa preparations, coffee and tea",milk and products thereof,cakes
562,foreign bodies,cereals and bakery products,plastic fragment,cakes
563,allergens,cereals and bakery products,peanuts and products thereof,cakes


In [ ]:
df1 = df_shifted.copy()
df1

,hazard-category,product-category,hazard,product
0,biological,"meat, egg and dairy products",listeria monocytogenes,chicken based products
1,biological,"meat, egg and dairy products",escherichia coli,ground beef
2,biological,"meat, egg and dairy products",escherichia coli,cheese
3,allergens,ices and desserts,peanuts and products thereof,ice cream
4,foreign bodies,"meat, egg and dairy products",metal fragment,chicken based products
...,...,...,...,...
560,allergens,fruits and vegetables,peanuts and products thereof,cakes
561,allergens,"cocoa and cocoa preparations, coffee and tea",milk and products thereof,cakes
562,foreign bodies,cereals and bakery products,plastic fragment,cakes
563,allergens,cereals and bakery products,peanuts and products thereof,cakes


In [ ]:
df2 = df_shifted.copy()
df2

,hazard-category,product-category,hazard,product
0,biological,"meat, egg and dairy products",listeria monocytogenes,chicken based products
1,biological,"meat, egg and dairy products",escherichia coli,ground beef
2,biological,"meat, egg and dairy products",escherichia coli,cheese
3,allergens,ices and desserts,peanuts and products thereof,ice cream
4,foreign bodies,"meat, egg and dairy products",metal fragment,chicken based products
...,...,...,...,...
560,allergens,fruits and vegetables,peanuts and products thereof,cakes
561,allergens,"cocoa and cocoa preparations, coffee and tea",milk and products thereof,cakes
562,foreign bodies,cereals and bakery products,plastic fragment,cakes
563,allergens,cereals and bakery products,peanuts and products thereof,cakes


In [ ]:
#drop hazard and product coloumn and save it as 'subtask1_predictions'
df1 = df1.drop(['hazard', 'product'], axis=1)
df1.to_csv('subtask1_predictions.csv', index=True)

#drop hazard and product coloumn and save it as 'subtask2_predictions'
df2 = df2.drop(['hazard-category', 'product-category'], axis=1)
df2.to_csv('subtask2_predictions.csv', index=True)